### 4. Noise Simulations

Now that we have simulated the clean sky signal (previous notebook) we need to create a noise signal as well. Simulated Signal + Simulated Noise should be a realistic model of what the instrument really sees.

The WMAP noise is specially simple: it's basically white noise, i.e., realizations from a standard gaussian distribution. 

The amplitude of the noise (the standard deviation of the gaussian) is proportional to $1/\sqrt{n}$, being $n$ the number of effective measurements on each sky region (pixel). So we need to load WMAP data to simulate the noise.

We make 100 noise realizations

In [ ]:
import numpy as np
import healpy as hp
from tqdm import tqdm  # package for progress bars in loops

In [ ]:
datapath = '../data/maps/wmap_iqumap_r9_9yr_K1_v5.fits'

wmap_k, wmap_header= hp.read_map(datapath, field=[0,1,2,3], h=True)
wmap_k_ii, wmap_header_ii = hp.read_map(datapath, hdu=2, field=[0,1,2,3], h=True)

nside = 512
npix = hp.nside2npix(nside) 

sigma0i_wmap , sigma0qu_wmap= 1.429, 1.435 # in mK

# Number of observations for intensity and polarization 
nobs_i =  wmap_k[3]
nobs_q = wmap_k_ii[1]
nobs_u = wmap_k_ii[3]
nobs_qu = wmap_k_ii[2]


# We have to take into account the mixing between the $Q$ and $U$ modes to get an effective count for $Q$ and $U$. 
neff_q = nobs_q - nobs_qu**2/nobs_u
neff_u = nobs_u - nobs_qu**2/nobs_q

sigma_i = sigma0i_wmap / np.sqrt(nobs_i)
sigma_q = sigma0qu_wmap / np.sqrt(neff_q)
sigma_u = sigma0qu_wmap / np.sqrt(neff_u)

noise_map = np.zeros([3,npix])

Make the noise realizations

In [6]:
for ii in tqdm(range(100), desc='generating maps'): 
    noise_map[0] = np.random.normal(0, sigma_i, npix)
    noise_map[1] = np.random.normal(0, sigma_q, npix)
    noise_map[2] = np.random.normal(0, sigma_u, npix)

    hp.write_map('../results/noise_simulations/wmap_k_noise_' + str(ii+1).zfill(4) + '.fits', noise_map, dtype=np.float64, overwrite = True)

generating maps: 100%|██████████| 100/100 [00:44<00:00,  2.23it/s]
